#  Local Regression Results

This notebook evaluates local regression under 2 factors:
- Kernel type: RBF and Laplacian (positive, distance-based kernels suitable for weighting)
- Dimensionality reduction: with vs without PCA

Evaluation :
1. Hold out a test set once (never used for tuning to prevent leakage).
2. Tune hyperparameters (γ) using k-fold CV on the remaining dev set.
3. Refit on the full dev set and evaluate once on the held-out test set.

Outputs:
- `results/tables/lr_results.csv`
- Figures under `results/figures/local_regression/`

In [ ]:
# Imports + path setup

import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.models.local_regression import LocalRegression
from src.kernels.rbf import RBFKernel
from src.kernels.laplacian import LaplacianKernel
from src.config import SEED, TEST_SIZE, K_FOLDS
from src.evaluation.splits import (train_test_split_indices, kfold_indices)
from src.evaluation.metrics import (rmse, mae)

# Make project root importable (assumes notebook is in ./notebooks)
PROJECT_ROOT = Path.cwd().resolve().parents[0]
if (PROJECT_ROOT / "src").exists():
    sys.path.insert(0, str(PROJECT_ROOT))

# Paths
DATA_DIR = PROJECT_ROOT / "data"
RESULTS_DIR = PROJECT_ROOT / "results"
TABLES_DIR = RESULTS_DIR / "tables"
FIG_DIR = RESULTS_DIR / "figures" / "local_regression"

TABLES_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
np.random.seed(SEED)

GAMMAS = [1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2,
          1e-1, 3e-1, 1.0, 3.0, 10.0, 30.0, 100.0]


In [ ]:
# Load preprocessed data

X_scaled = np.load(DATA_DIR / "X_scaled.npy")
X_pca = np.load(DATA_DIR / "X_pca.npy")

# labels may be y.npy or t.npy
if (DATA_DIR / "y.npy").exists():
    y = np.load(DATA_DIR / "y.npy")
else:
    y = np.load(DATA_DIR / "t.npy")

assert X_scaled.shape[0] == y.shape[0]
assert X_pca.shape[0] == y.shape[0]

print("X_scaled:", X_scaled.shape)
print("X_pca:", X_pca.shape)
print("y:", y.shape)
